# **Implementación del cifrado Hill en Python**
_Juan Esteban Alarcón Bravo_

## 1. Llave y Mensaje

Se inicia el procedimiento solicitando al usuario el ***mensaje*** (`msg`) y la ***llave*** (`key`). El mensaje se pone todo en mayúsculas, se eliminan los caracteres de espacio (en caso de tenerlos) y se rellena con $X$ hasta que su longitud sea un número par:

In [1]:
msg = str(input("Ingrese el mensaje: "))
key = str(input("Ingrese la llave: "))

msg = msg.upper()
msg = msg.replace(" ", "")

key = key.split(',')
key = list(map(int, key))

# Relleno de msg con "X"
while len(msg) % 2 != 0:
  msg += "X"

msgLen = len(msg)

Ingrese el mensaje: Original Plaintext
Ingrese la llave: 2,3,3,6


## 2. Inicializacion de Variables y Definición de Funciones Auxiliares

_**Variables:**_
* `keyMatrix` es la matriz de `key`. Es una matriz $M_{2\times2}$.
* `invKeyMatrix` es la matriz inversa de `key` $\mod 26$. Es una matriz $M_{2\times2}$.
* `msgVector` es el vector de `msg`. Es un vector columna $V_{msgLen \times 1}$.
* `outVector` es el vector resultante de cifrar o descifrar `msg`. Tiene las mismas dimensiones de `msgVector`.
* `cipherText` es el string del mensaje cifrado o descifrado obtenido.

In [2]:
keyMatrix = [[0] * 2 for i in range(2)]
invKeyMatrix = [[0] * 2 for i in range(2)]
msgVector = [[0] for i in range(msgLen)]
outVector = [[0] for i in range(msgLen)]
cipherText = []

_**Funciones:**_
* `getKeyMatrix()` convierte el string original de `key` en una matriz de $2 \times 2$. Este valor se le asigna a `keyMatrix`.
* `inverseMatrix()` invierte la llave módulo 26 utilizando `modInv()` y `redux()`.

In [3]:
import numpy as np

def getKeyMatrix(key):
  k = 0
  for i in range(2):
    for j in range(2):
      # keyMatrix[i][j] = ord(key[k]) % 65
      keyMatrix[i][j] = key[k]
      k += 1

def inverseMatrix(A,p):
  n = len(A)
  A = np.matrix(A)
  adj = np.zeros(shape=(n,n))
  for i in range(0,n):
    for j in range(0,n):
      adj[i][j] = ((-1)**(i+j)*int(round(np.linalg.det(redux(A,j,i))))) % p
  return (modInv(int(round(np.linalg.det(A))),p)*adj) % p

def modInv(a,p):
	for i in range(1,p):
		if (i*a)%p==1: return i
	raise ValueError("La llave no tiene inverso mod " + str(p))
 
def redux(A,i,j):
  A = np.array(A)
  redux = np.zeros(shape = (len(A)-1,len(A)-1))
  p = 0
  for s in range(0,len(redux)):
    if p==i: p=p+1
    q=0
    for t in range(0,len(redux)):
      if q==j: q=q+1
      redux[s][t]=A[p][q]
      q=q+1
    p=p+1
  return redux

## 3. Encripción y Desencripción

Suponemos que se está usando un alfabeto de 26 caracteres ($ABCDEFGHIJKLMNOPQRSTUVWXYZ$).

Para encriptar, se realiza la operación:
$$
keyMatrix \times msgVector = outVector
$$

In [4]:
def encrypt(msgVector, cipherText):
  for i in range(2):
    for j in range(1):
      outVector[i][j] = 0
      for x in range(2):
        outVector[i][j] += keyMatrix[i][x] * msgVector[x][j]
      outVector[i][j] = outVector[i][j] % 26
    cipherText.append(chr(outVector[i][0] + 65))

Para la desencripción, por su parte, basta con cambiar la matriz de la llave por su inversa y realizar la misma multiplicación:
$$
invKeyMatrix \times msgVector = outVector
$$

In [5]:
def decrypt(msgVector, cipherText):
  for i in range(2):
    for j in range(1):
      outVector[i][j] = 0
      for x in range(2):
        outVector[i][j] += int(invKeyMatrix[i][x]) * msgVector[x][j]
      outVector[i][j] = outVector[i][j] % 26
    cipherText.append(chr(outVector[i][0] + 65))

In [6]:
# Generación de la matriz de la llave y su inversa modular:
getKeyMatrix(key)
invKeyMatrix = inverseMatrix(keyMatrix,26)

# Vectorización de msg
for i in range(msgLen):
    msgVector[i][0] = ord(msg[i]) % 65

## 4. Impresión de los resultados

In [7]:
print("CIFRADO HILL (2x2) \n¿Qué desea hacer?")
choice = int(input(" 1. Encriptar \n 2. Desencriptar \n"))

if choice == 1:
  print("\n---------------\n\nMensaje Original: " + msg)
  print("Llave: " + str(keyMatrix))
  for x in range(0,msgLen,2):
    encrypt(msgVector[x:x+2], cipherText)
  print("\nMensaje Cifrado: ", "".join(cipherText))
elif choice == 2:
  print("\n---------------\n\nMensaje Cifrado: " + msg)
  print("Llave: " + str(keyMatrix))
  for x in range(0,msgLen,2):
    decrypt(msgVector[x:x+2], cipherText)
  print("\nMensaje Descifrado: ", "".join(cipherText))

CIFRADO HILL (2x2) 
¿Qué desea hacer?
 1. Encriptar 
 2. Desencriptar 
1

---------------

Mensaje Original: ORIGINALPLAINTEXTX
Llave: [[2, 3], [3, 6]]

Mensaje Cifrado:  BOIIDYHOLHYWFXZUDN
